In [2]:
from zipfile import ZipFile
import sys
sys.path.insert(0, '/mnt/d/Documents/Chores/homework/NSS/main/lab/sslc/signer/sm-sign')
import io
dir = "/mnt/d/Documents/Chores/homework/NSS/main/lab/sslc/test/valid/"
file_name = "com.android.vending-16.apk"
import sm
import os.path
import base64
sign_apk = ZipFile(os.path.join(dir, file_name) + ".sm2", mode = 'w')
key = sm.generate_SM2_key_files(b'pubkey.ec', b'pri_key.ec', b'12345678')
with ZipFile(os.path.join(dir, file_name), 'r') as zfile:
    mf = b'Manifest-Version: 1.0\nBuilt-By: Sign tool\nCreated-By: Siliang Qin\n'
    dgst = b''
    for finfo in zfile.infolist():
        if('META-INF' in finfo.filename):
            continue
        content = zfile.read(finfo.filename)
        sign_apk.writestr(finfo.filename, content)
        s = sm.SM3(content)
        dgst += b'\nNAME: %s\nSM3-Digest: %s\n' %( finfo.filename.encode('utf-8'), base64.b64encode(s))
    mf += dgst        
    sign_apk.writestr('META-INF/MANIFEST.MF', mf)
    sf = b"Signature-Version: 1.0\nX-Android-APK-Signed: 2\n"
    sf += b"SM3-Digest-Manifest: "
    sf += base64.b64encode(sm.SM3(mf))
    sf +=b'\nCreated-By: 1.0 (Android)\n'
    sf += dgst
    sign_apk.writestr('META-INF/CERT.SF', sf)
    key = sm.read_private_key(b'pri_key.ec', b'12345678')
    pubkey = sm.read_public_key(b'pubkey.ec')
    # sign = sm.SM2_SIGN(sf, key)
    x509 = sm.generate_x509(key, pubkey, b'cert.x509')
    p7 = sm.pkcs7_sign(x509, key, sf)
    # print(p7)
    sign_apk.writestr('META-INF/CERT.SM2', p7)
    s = sm.pkcs7_verify(p7, sf)
    print(s)
    # print(sign)
sign_apk.close()

False


In [3]:
key = sm.generate_SM2_key_files(b'pubkey.ec', b'pri_key.ec', b'12345678')

In [3]:
priv_key = sm.read_private_key(b'pri_key.ec', b'12345678')

In [4]:
pkcs8 = sm.write_pkcs8(b'key.pk8', priv_key, b'12345678')

In [7]:
x509 = sm.generate_x509(priv_key, priv_key, b'cert.x509')